In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import linregress
from sklearn.impute import KNNImputer as KNN

params = {'legend.fontsize': 18,
          'figure.figsize': (13, 10),
         'axes.labelsize': 24,
         'axes.titlesize':24,
         'axes.linewidth':5,
         'xtick.labelsize':20,
         'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

# Import data

In [ ]:
path = '/Users/sjiang87/AtmosphericData/data/Hourly_data_of_Beijing_from_Jinxi_interpolated.csv'
df = pd.read_csv(path)
df.head()

# Interpolate missing values and save separate .csv 

In [ ]:
'''# define feature data to interpolate 
features = ['CO', 'NO2', 'O3', 'PM10', 'PM2.5', 'SO2', 'ISD_t2m', \
       'ISD_d2m', 'ISD_sp', 'ISD_rh', 'ISD_u', 'ISD_v', 'ERA5_d2m', 'ERA5_t2m', \
       'ERA5_rh', 'ERA5_sp', 'ERA5_u10', 'ERA5_v10', 'ERA5_blh']

# Use 5 nearest rows which have a feature to fill in each row's missing features
X_incomplete = np.array(df[features].values, np.float32)
X_filled_knn = KNN(n_neighbors=5).fit_transform(X_incomplete)
df_filled = df.copy()
for j,feature in enumerate(features):
    df_filled[feature] = X_filled_knn[:, j]

# save interpolated data
path = 'data/Hourly_data_of_Beijing_from_Jinxi_interpolated.csv'
df_filled.to_csv(path, index=False)'''
print("Already interpolated data.")

# Define input and target variables

In [ ]:
# define some random variables to track over time

# inputs should include time and holiday as inputs, but these need to be encoded somehow
# inputs = ['Time', 'Holiday', 'ERA5_d2m', 'ERA5_t2m', 'ERA5_rh', 'ERA5_sp', 'ERA5_u10', 'ERA5_v10', 'ERA5_blh']
inputs = ['ERA5_d2m', 'ERA5_t2m', 'ERA5_rh', 'ERA5_sp', 'ERA5_u10', 'ERA5_v10', 'ERA5_blh']
targets = ['SO2', 'PM2.5']
df[targets]

In [ ]:
plt.hist(df[targets[0]].values)
plt.yscale('log')
plt.title(targets[0])

In [ ]:
plt.hist(df[targets[1]].values)
plt.title(targets[1])
plt.yscale('log')

# Example 1: Use DMD to model only target variables at single site

# Pick a site and pull X matrix

In [ ]:
df = pd.read_csv(path)
site_num = 0

sites = df['Site'].values 
unique_sites = np.unique(sites)
inds = sites == unique_sites[site_num]
df = df[targets].iloc[inds].interpolate().copy()
print(f"Looking only at site {unique_sites[site_num]}")
df.head()

# Basic DMD model

In [ ]:
# Define Xt: shape m x n, m = number of features, n = number of time points
Xt = np.array(df.values.T, np.float32)
m, n = Xt.shape
X  = Xt[:, :-1]
Xp = Xt[:, 1:]

# solve for A where AX ~= Xp 
A = Xp @ np.linalg.pinv(X)

# Plot random trajectory from training data

In [ ]:
def predict(A, X0, n):
    Xpred = np.zeros([A.shape[0], n])
    Xpred[:, 0] = X0
    for i in range(1, n):
        Xpred[:, i] = A@Xpred[:, i-1]
    return Xpred

# random starting point
n_start = np.random.choice(np.arange(n))
n_pred  = 24 # hours to predict
Xpred   = predict(A, X[:, n_start], n_pred)
Xtrue   = Xp[:, n_start-1:n_start+n_pred-1]

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    plt.plot(Xpred[i], c=f'C{i}', label=f'Predicted')
    plt.plot(Xtrue[i], c=f'C{i}', label=f'True', linestyle='--')
    plt.title(f"Prediction of {targets[i]}")
    plt.xlabel("hours ahead")
    plt.legend()
plt.suptitle("Fit to data\n", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.savefig("Figures/DMD_example_2.png")
plt.show()

In [ ]:
# compute reconstruction of Xp
Xr = A@X

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    R = linregress(Xr[i], Xp[i]).rvalue
    plt.scatter(Xr[i], Xp[i], label='R: {:.3f}'.format(R))
    plt.title(f"Prediction of {targets[i]}")
    plt.legend()
plt.suptitle("Fit to data\n", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#plt.savefig("Figures/DMD_example_1.png", dpi=100)
plt.show()

# Example 2: Augmented DMD model

In [ ]:
# Define function to create augmented matrix 
def augment(Xt, l):
    m, n = Xt.shape
    Xt_aug = np.zeros([m*l, n-l+1])
    for i in range(l):
        Xt_aug[i*m:(i+1)*m] = Xt[:, i:n-l+i+1]
    X_aug  = Xt_aug[:, :-1]
    Xp_aug = Xt_aug[:, 1:] 
    return Xt_aug, X_aug, Xp_aug

# Define Xt: shape m x n, m = number of features, n = number of time points
l  = 12
Xt = np.array(df.values.T, np.float32)
m, n = Xt.shape
X  = Xt[:, :-1]
Xp = Xt[:, 1:]
Xt_aug, X_aug, Xp_aug = augment(Xt, l)

# solve for A where AX ~= Xp 
A_aug = Xp_aug @ np.linalg.pinv(X_aug)

# Plot random trajectory from training data

In [ ]:
def predict_aug(A, X, m, n):
    Xpred = np.zeros([m, n])
    Xpred[:, 0] = np.copy(X[-m:])
    for i in range(1, n):
        X = A@X 
        Xpred[:, i] = np.copy(X[-m:])
    return Xpred

# random starting point
n_start = np.random.choice(np.arange(n))
n_pred  = 24 # hours to predict
Xpred   = predict_aug(A_aug, X_aug[:, n_start], m, n_pred)
Xtrue   = X_aug[-m:, n_start:n_start+n_pred-1]

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    plt.plot(Xpred[i], c=f'C{i}', label=f'Predicted')
    plt.plot(Xtrue[i], c=f'C{i}', label=f'True', linestyle='--')
    plt.title(f"Prediction of {targets[i]}")
    plt.xlabel("hours ahead")
    plt.legend()
plt.suptitle(f"Fit to data (l={l})", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
l_list = [1, 12, 24]

Xt = np.array(df.values.T, np.float32)
m, n = Xt.shape
X  = Xt[:, :-1]
Xp = Xt[:, 1:]

plt.figure(figsize=(14, 10))
k = 0
for l in l_list:
    
    Xt_aug, X_aug, Xp_aug = augment(Xt, l)

    # solve for A where AX ~= Xp 
    A_aug = Xp_aug @ np.linalg.pinv(X_aug)
    # compute reconstruction of Xp
    Xr = (A_aug@X_aug)[-m:, :]

    for i in range(m):
        plt.subplot(3,2,k+1)
        k += 1
        R = linregress(Xr[i], Xp_aug[-m:, :][i]).rvalue
        plt.scatter(Xr[i], Xp_aug[-m:, :][i], label='R: {:.3f}'.format(R))
        plt.title(f"Prediction of {targets[i]}, l={l}")
        plt.legend(loc='upper left')
        
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("Figures/DMD_example_3.png", dpi=100)
plt.show()

# Example 3: DMDc model $x_{k+1} = Ax_{k} + Bu_{k}$

In [ ]:
df = pd.read_csv(path)
site_num = 0

sites = df['Site'].values 
unique_sites = np.unique(sites)
inds = sites == unique_sites[site_num]
dfX = df[targets].iloc[inds].copy()
dfU = df[inputs].iloc[inds].copy()
print(f"Looking only at site {unique_sites[site_num]}")

# Set up matrices Xp, X, L 
Xt = np.array(dfX.values.T, np.float32)
Lt = np.array(dfU.values.T, np.float32)

# standardize output 
Xt = Xt.T
Xt = (Xt - np.mean(Xt, 0)) / np.std(Xt, 0)
Xt = Xt.T

# standardize control input 
Lt = Lt.T
Lt = (Lt - np.mean(Lt, 0)) / np.std(Lt, 0)
Lt = Lt.T

m, n = Xt.shape
X  = Xt[:, :-1]
L  = Lt[:, :-1]
Xp = Xt[:, 1:]

def DMDc(Xp, X, L):
    n = X.shape[0]
    l = L.shape[0]
    # Gamma matrix = [X; L]
    Gamma = np.concatenate((X, L))
    U, S, Vh = np.linalg.svd(Gamma, full_matrices=False)
    U1 = U[:n, :]
    U2 = U[n:, :]
    A = Xp@Vh.T/S@U1.T
    B = Xp@Vh.T/S@U2.T
    return A, B

# run DMD w/ control!
A, B = DMDc(Xp, X, L)

# Plot trajectory using DMDc 

In [ ]:
def predict(A, B, X0, U, n):
    Xpred = np.zeros([A.shape[0], n])
    Xpred[:, 0] = X0
    for i in range(1, n):
        Xpred[:, i] = A@Xpred[:, i-1] + B@U[:, i-1]
    return Xpred

# random starting point
n_start = np.random.choice(np.arange(n))
n_pred  = 24 # hours to predict
Xpred   = predict(A, B, X[:, n_start], L[:, n_start:(n_start+n_pred)], n_pred)
Xtrue   = X[:, n_start:n_start+n_pred]

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    plt.plot(Xpred[i], c=f'C{i}', label=f'Predicted')
    plt.plot(Xtrue[i], c=f'C{i}', label=f'True', linestyle='--')
    plt.title(f"Prediction of {targets[i]}")
    plt.xlabel("hours ahead")
    plt.legend()
plt.suptitle("Fit to data\n", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#plt.savefig("Figures/DMD_example_5.png")
plt.show()

# DMDc reconstruction 

In [ ]:
# compute reconstruction of Xp
Xr = A@X + B@L

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    R = linregress(Xr[i], Xp[i]).rvalue
    plt.scatter(Xr[i], Xp[i], label='R: {:.3f}'.format(R))
    plt.title(f"Prediction of {targets[i]}")
    plt.legend()
plt.suptitle("Fit to data\n", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
#plt.savefig("Figures/DMD_example_4.png", dpi=100)
plt.show()

# Analyze B matrix

In [ ]:
for i, target in enumerate(targets):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.bar(range(L.shape[0]), B[i, :], align='center')
    ax.set_xticks(range(L.shape[0]))
    ax.set_xticklabels(inputs)
    plt.xticks(rotation=45)
    plt.ylabel("Control coefficient")
    plt.title(f"Control influence on {target}")
    plt.savefig(f"DMD_ControlEffect_{target}.png", dpi=100)
    plt.show()

# Example 4: DMDc with delay embedding

In [ ]:
def DMDc(Xp, X, L):
    n = X.shape[0]
    l = L.shape[0]
    # Gamma matrix = [X; L]
    Gamma = np.concatenate((X, L))
    U, S, Vh = np.linalg.svd(Gamma, full_matrices=False)
    U1 = U[:n, :]
    U2 = U[n:, :]
    A = Xp@Vh.T/S@U1.T
    B = Xp@Vh.T/S@U2.T
    return A, B

# Define function to create augmented matrix 
def augment(Xt, l):
    m, n = Xt.shape
    Xt_aug = np.zeros([m*l, n-l+1])
    for i in range(l):
        Xt_aug[i*m:(i+1)*m] = Xt[:, i:n-l+i+1]
    X_aug  = Xt_aug[:, :-1]
    Xp_aug = Xt_aug[:, 1:] 
    return Xt_aug, X_aug, Xp_aug

# import data
df = pd.read_csv(path)
site_num = 0
sites = df['Site'].values 
unique_sites = np.unique(sites)
inds = sites == unique_sites[site_num]
dfX = df[targets].iloc[inds].copy()
dfU = df[inputs].iloc[inds].copy()
print(f"Looking only at site {unique_sites[site_num]}")

# Define time lag parameter (hrs)
l  = 12

# Set up matrices Xp, X, L 
Xt = np.array(dfX.values.T, np.float32)
Lt = np.array(dfU.values.T, np.float32)
m, n = Xt.shape
Xt_aug, X_aug, Xp_aug = augment(Xt, l)
Lt_aug, L_aug, Lp_aug = augment(Lt, l)

# run DMD w/ control!
A, B = DMDc(Xp_aug, X_aug, L_aug)
[s.shape for s in [Xt_aug, X_aug, Xp_aug, Lt_aug, L_aug, Lp_aug]]

# Plot trajectory

In [ ]:
def predict_aug(A, B, X, L, m, n):
    Xpred = np.zeros([m, n])
    Xpred[:, 0] = np.copy(X[-m:])
    for i in range(1, n):
        X = A@X + B@L[:, i]
        Xpred[:, i] = np.copy(X[-m:])
    return Xpred

# random starting point
n_start = np.random.choice(np.arange(n))
n_pred  = 24 # hours to predict
Xpred   = predict_aug(A, B, X_aug[:, n_start], L_aug[:, n_start:n_start+n_pred], m, n_pred)
Xtrue   = X_aug[-m:, n_start:n_start+n_pred-1]
print([Xpred.shape, Xtrue.shape])

plt.figure(figsize=(14, 6))
for i in range(m):
    plt.subplot(1,2,i+1)
    plt.plot(Xpred[i], c=f'C{i}', label=f'Predicted')
    plt.plot(Xtrue[i], c=f'C{i}', label=f'True', linestyle='--')
    plt.title(f"Prediction of {targets[i]}")
    plt.xlabel("hours ahead")
    plt.legend()
plt.suptitle(f"Fit to data (l={l})", fontsize=24)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.savefig("Figures/DMD_example_7.png", dpi=100)
plt.show()

In [ ]:
# define list of time lags to try 
l_list = [1, 12, 24]

# Set up matrices Xp, X, L 
Xt = np.array(dfX.values.T, np.float32)
Lt = np.array(dfU.values.T, np.float32)
m, n = Xt.shape

plt.figure(figsize=(14, 10))
k = 0
for l in l_list:
    # augment X and L
    Xt_aug, X_aug, Xp_aug = augment(Xt, l)
    Lt_aug, L_aug, Lp_aug = augment(Lt, l)

    # run DMD w/ control!
    A, B = DMDc(Xp_aug, X_aug, L_aug)
    
    # compute reconstruction of Xp
    Xr = (A@X_aug + B@L_aug)[-m:, :]

    for i in range(m):
        plt.subplot(3,2,k+1)
        k += 1
        R = linregress(Xr[i], Xp_aug[-m:, :][i]).rvalue
        plt.scatter(Xr[i], Xp_aug[-m:, :][i], label='R: {:.3f}'.format(R))
        plt.title(f"Prediction of {targets[i]}, l={l}")
        plt.legend(loc='upper left')
        
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.savefig("Figures/DMD_example_6.png", dpi=100)
plt.show()

# Validate model on held-out data